# Assignment 5 
by [Jay Nakum](https://jaynakum.github.io) [20BCP304D]

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
heart=pd.read_csv('./datasets/heart.csv')

In [ ]:
heart

In [ ]:
heart.describe()

In [ ]:
heart.nunique()

In [ ]:
heart.isnull().any()

In [ ]:
heart.info()

In [ ]:
#negligible correlation between numerical variables
plt.figure(figsize= (16, 8))
sns.heatmap(heart.corr(), annot = True, fmt= '.2f');

In [ ]:
heart['output'].value_counts()

In [44]:
categorical_cols=['sex','cp','fbs','restecg','exng','slp','caa','thall','output']

normalizing the numerical features

In [ ]:
numerical_cols=np.setdiff1d(heart.columns,categorical_cols)
numerical_cols

In [46]:
heart[numerical_cols]=(heart[numerical_cols]-heart[numerical_cols].mean())/heart[numerical_cols].std()

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
x=heart.drop('output',axis=1)
y=heart['output']

In [49]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y)

In [ ]:
xtrain.shape

In [ ]:
xtest.shape

In [52]:
from sklearn.svm import SVC

In [53]:
from feature_engine.selection import DropConstantFeatures,DropDuplicateFeatures,SmartCorrelatedSelection
from sklearn.pipeline import Pipeline
pipe=Pipeline([('constant',DropConstantFeatures(tol=0.95)),('duplicate',DropDuplicateFeatures()),('smartcorrelation',SmartCorrelatedSelection(threshold=0.95,estimator=SVC))])

In [54]:
xtrainnew=pipe.fit_transform(xtrain)

In [ ]:
xtrainnew.shape #no feature was dropped

Performing grid search

In [57]:
svm=SVC(random_state=0)

In [58]:
from sklearn.model_selection import GridSearchCV

In [59]:
#using rbf kernel
parameters=[{'gamma':[0.01,0.1,0.5,0.8,1,10],'C':[int(i) for i in np.linspace(start=1,stop=100,num=11)]}]
grid=GridSearchCV(svm,param_grid=parameters,cv=4,verbose=1)

In [60]:
grid.fit(xtrain,ytrain)

Fitting 4 folds for each of 66 candidates, totalling 264 fits


GridSearchCV(cv=4, estimator=SVC(random_state=0),
             param_grid=[{'C': [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                          'gamma': [0.01, 0.1, 0.5, 0.8, 1, 10]}],
             verbose=1)

In [61]:
grid.best_estimator_

SVC(C=20, gamma=0.01, random_state=0)

In [62]:
grid.cv_results_

{'mean_fit_time': array([0.00774324, 0.00799614, 0.00749803, 0.0064981 , 0.00900048,
        0.00749052, 0.00425053, 0.00500137, 0.00649375, 0.00699979,
        0.00650012, 0.01024175, 0.00474977, 0.00649762, 0.0067538 ,
        0.00575107, 0.00625139, 0.0079971 , 0.00549716, 0.00574762,
        0.00773948, 0.00574696, 0.00624436, 0.00774568, 0.00499666,
        0.00549597, 0.00724894, 0.00524706, 0.00599778, 0.00674802,
        0.00500101, 0.00549912, 0.00549668, 0.00599325, 0.00500131,
        0.00699353, 0.00474793, 0.00474989, 0.00549763, 0.00549614,
        0.00524735, 0.00774711, 0.00449872, 0.00549918, 0.00575018,
        0.00524986, 0.00599611, 0.00699639, 0.00499541, 0.00524789,
        0.00549918, 0.00649738, 0.00524974, 0.00649536, 0.00524706,
        0.00550038, 0.00574994, 0.0062483 , 0.00499845, 0.00699812,
        0.00474656, 0.00499797, 0.00524664, 0.00499731, 0.00524777,
        0.00650179]),
 'std_fit_time': array([1.78936966e-03, 1.22665628e-03, 1.65790855e-03, 1.500

In [63]:
svmm=SVC(gamma=0.01,C=10,random_state=0)

In [64]:
svmm.fit(xtrain,ytrain)

SVC(C=10, gamma=0.01, random_state=0)

In [65]:
pred=svmm.predict(xtest)

In [66]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [67]:
accuracy_score(ytest,pred)

0.8026315789473685

In [68]:
confusion_matrix(ytest,pred)

array([[23, 11],
       [ 4, 38]], dtype=int64)

In [69]:
print(classification_report(ytest,pred))

              precision    recall  f1-score   support

           0       0.85      0.68      0.75        34
           1       0.78      0.90      0.84        42

    accuracy                           0.80        76
   macro avg       0.81      0.79      0.79        76
weighted avg       0.81      0.80      0.80        76

